In [1]:
from scipy.io import wavfile
import torch
import sounddevice as sd
import pytorch_lightning as pl
from trainer import Iminegg
from my_iterable_dataset import MyIterableDataset
from im_in_egg_unet1_param import ImineggNet1
from im_in_egg_unet1_param import ImineggNet1 as ImineggNet1FT
from torch.utils.data import DataLoader

C:\Users\alsa0217\Anaconda3\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [2]:
ds_validate = MyIterableDataset(100, "./data/val22", 22528 * 4 * 4)

c:\ML\iminegg\my_iterable_dataset.py:19: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  self.voices.append(torch.from_numpy(data).float() / 32768)


In [21]:
from im_in_egg_unet1_param_add3l_2 import ImineggNet1_1 as ImineggNet1

iminegg = Iminegg(ImineggNet1(32))
validate_set = DataLoader(ds_validate, batch_size=20)
ckpt = "epoch=503_no_speach_split.ckpt" 
ckpt = "D:\epoch=927_supremacy_of_noise.ckpt"
ckpt = "w48_2l_797.ckpt"
ckpt = "logs/defaultw32-bs40-drp0.2_relu0.2_add3l_33_17_9_5/version_0/checkpoints/epoch=441.ckpt"
trainer = pl.Trainer(resume_from_checkpoint=ckpt, gpus=0)
trainer.test(model=iminegg, test_dataloaders=validate_set, ckpt_path=ckpt)


GPU available: True, used: False
TPU available: False, using: 0 TPU cores


1

In [63]:
from im_in_egg_unet1_param_add3l import ImineggNet1 as ImineggNet

iminegg2 = Iminegg(ImineggNet(32))
validate_set = DataLoader(ds_validate, batch_size=20)
ckpt = "epoch=503_no_speach_split.ckpt" 
ckpt = "D:\epoch=927_supremacy_of_noise.ckpt"
ckpt = "w48_2l_797.ckpt"
ckpt = "logs/defaultw32-bs40-drp0.2_relu0.2_add2l/version_2/checkpoints/epoch=707.ckpt"
trainer2 = pl.Trainer(resume_from_checkpoint=ckpt, gpus=0)
trainer2.test(model=iminegg2, test_dataloaders=validate_set, ckpt_path=ckpt)
torch.save(iminegg2.state_dict(), 'defaultw32-bs40-drp0.2_relu0.2_add2l_707.model')



GPU available: True, used: False
TPU available: False, using: 0 TPU cores


In [61]:
from im_in_egg_unet1_param_norm import ImineggNet1 as ImineggNet3

iminegg3 = Iminegg(ImineggNet3(56))
ckpt = "epoch=503_no_speach_split.ckpt" 
ckpt = "D:\epoch=927_supremacy_of_noise.ckpt"
ckpt = "w48_2l_797.ckpt"
ckpt = "logs/defaultw56-bs50-drp0.0_relu0.2_NS/version_0/checkpoints/epoch=615.ckpt"
trainer3 = pl.Trainer(resume_from_checkpoint=ckpt, gpus=0)
trainer3.test(model=iminegg3, test_dataloaders=validate_set, ckpt_path=ckpt)
torch.save(iminegg3.state_dict(), 'defaultw56-bs50-drp0.0_relu0.2_NS_615.model')


GPU available: True, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
ckpt = "logs/defaultFT_w32-bs40-drp0.2_relu0.2/version_2/checkpoints/epoch=330.ckpt"
iminegg_net_ft = ImineggNet1FT(32)
imineggft = Iminegg(iminegg_net_ft)
trainer2 = pl.Trainer(resume_from_checkpoint=ckpt, gpus=0)
trainer2.test(model=imineggft, ckpt_path=ckpt)


In [4]:
samplerate, data = wavfile.read('./tests/source_17.wav')
data = (torch.from_numpy(data).float() / 32768)
sec1 = 11264
off = 0
length = (len(data) - off) // sec1 * sec1

inp = data[off:off + length]

In [39]:
d = next(iter(ds_validate))
inp = d['input']
print(d['type'])
device = torch.device("cpu")
samplerate = 22050

2


In [47]:
sd.play(inp.cpu().numpy(), samplerate)

In [60]:
%%time
trainer.model.net.eval()
trainer2.model.net.eval()
trainer3.model.net.eval()
outp = trainer2.model.net.calc(inp.reshape(1, 1, -1).to(device))
#outp = trainer.model.net.calc(outp.reshape(1, 1, -1).to(device))

Wall time: 3.48 s


In [59]:
sd.play(outp.cpu().detach().numpy().reshape(-1), samplerate)

In [64]:
export_outp = (outp.cpu() * 32768).short()
export_inp = (inp.cpu() * 32768).short()

wavfile.write('tests/filtered_22khz_21.wav', samplerate, export_outp.detach().numpy())
wavfile.write('tests/source_22khz_21.wav', samplerate, export_inp.detach().numpy())